In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re as re
import seaborn as sns
import datetime as dt
import openpyxl
import xlrd

In [ ]:
commune=pd.read_excel(r"commune.xlsx")
enroll = pd.read_csv(r"enroll.csv")
quest = pd.read_csv(r"quest.csv")
industry = pd.read_csv(r"industry.csv")
ord = pd.read_csv(r"ord.csv")
study_domain = pd.read_csv(r"study_domain.csv")
transaction = pd.read_csv(r"transaction.csv")

In [ ]:
def dummies_create(data, value):
  x=pd.get_dummies(data=data, columns=value, prefix="", prefix_sep="")
  y=x.groupby("quest_id").sum()
  return y

In [ ]:
study_domain2 = dummies_create(study_domain[["key", "quest_id", "values"]], ["values"])
technology2 = dummies_create(technology[["key", "quest_id", "values"]], ['values'])
industry2 = dummies_create(industry[["key", "quest_id", "values"]], ['values'])

In [ ]:
quest.drop(['Unnamed: 0', 'modified_at',"study_domain", 'department'], axis=1, inplace = True)
quest.rename(columns={'created_at':"signup_date", 'hear_AA_1': "chanels", "after_AA":"Project" },inplace = True )
quest["signup_date"] = quest["signup_date"].str[:10]
print(f"The number of appliants in the bootcamp is {len(quest)}.")

In [ ]:
quest["dob"].replace({"3 aout 1977": "3/08/1977"}, inplace=True)
quest["dob"] = pd.to_datetime(quest["dob"])
quest["signup_date"] = pd.to_datetime(quest["signup_date"])

In [ ]:
quest["age"] = (-(quest["dob"]-quest["signup_date"])//365)
quest["age"] = quest["age"].apply(lambda x: str(x)[:2])
age=quest["age"].tolist()
age = [0 if val == "Na" else val for val in age]
age = [int(val) for val in age]
age = [sum(age)/len(age) if (val == 0 or val == 1.0 or val ==-1.0) else val for val in age]
quest["age"]= age

In [ ]:
ord["paid"] = "Carte"
transaction["paid"] = "MonCash"
print(f"{len(enroll)} people enrolled in the course Introduction to Data Science, {len(ord)} paid by credit card and {len(transaction)} paid by moncash." )
print("We notice that not all enrolled necessarly apply for the bootcamp.")

In [ ]:
data = pd.merge(left = enroll, right = ord, how="left", on="quest_id")
#dta = pd.merge(left = data, right = transaction, how="left", on="user_id")
#d = pd.merge(left=quest, right=dta, how = "left", on="quest_id")
#print(data.info())

In [ ]:
commune["Commune_Id"] = commune["Commune_Id"].str.lower()
commune.rename(columns = {"Commune_Id": "commune"}, inplace = True)

In [ ]:
result=pd.merge(left=quest, right=enroll[["percentage_completed","completed", "quest_id"]], how = 'left',on='quest_id')
result=pd.merge(left=result, right=study_domain2, how = 'left',on='quest_id')
result=pd.merge(left=result, right=technology2, how = 'left',on='quest_id')
result=pd.merge(left=result, right=industry2, how = 'left',on='quest_id')
result=pd.merge(left=result, right=commune[["Commune_FR","Departement", "commune"]], how = 'left',on='commune')
result.head(2)

In [ ]:
ord.head(2)
ord1 = ord.rename(columns = {'created_at': 'createdord'}, inplace = True)
transaction.rename(columns = {'created_at': 'createdtrans'}, inplace = True)
print(transaction.columns)
ord.head(2)

In [ ]:
result["male"] = result["gender"].apply(lambda x: 1 if x == "male" else 0)
result["female"] = result["gender"].apply(lambda x: 1 if x == "female" else 0)
gender = pd.DataFrame()
gender["male"]=[result["male"].sum()]
gender["female"]=[result["female"].sum()]
gender

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data = quest,x=quest["age"],kde=True,bins=20)
plt.title("Age distribution")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=quest,x="gender",y="age")
#quest[quest["age"]<10]

In [ ]:
technology1 = technology2.reset_index()
tech1 = technology1.drop(["quest_id"], inplace=False, axis=1).sum(axis=0)
tech1 = tech1.sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(y=tech1.index, x = tech1)
plt.title("The knowledge of technologies used by participants")
plt.xlabel("Occurence")
plt.ylabel("Technologies")

In [ ]:
industry1 = industry2.reset_index()
indus1 = industry1.drop(["quest_id"], inplace=False, axis=1).sum(axis=0)
indus1 = indus1.sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(y=indus1.index, x = indus1)
plt.title("The domain of work by participants")
plt.xlabel("Occurence")
plt.ylabel("Domains")

In [ ]:
study_domain1 = study_domain2.reset_index()
stud1 = study_domain1.drop(["quest_id"], inplace=False, axis=1).sum(axis=0)
stud1 = stud1.sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(y=stud1.index, x = stud1)
plt.title("The domains of study by participants")
plt.xlabel("Occurence")
plt.ylabel("Domains of study")

In [ ]:

def generate_barchart(data="", title ="",abs_value ="Total",rel_value="Percent",figsize =(7,3),horizontal=""):
  plt.figure(figsize=figsize)
  plt.style.use("ggplot")
  axes = sns.barplot(data=data,x=data.index,y=abs_value)
  i=0
  for tot, perc in zip(data[abs_value],data[rel_value]):
    axes.text(i,
              tot/2,
              str(np.round(perc*100,2))+ "%",
               fontdict=dict(color='White',fontsize=12,horizontalalignment="center")
              )
    axes.text(i,
              tot+ 3,
              str(tot),
               fontdict=dict(color='blue',fontsize=12,horizontalalignment="center")
              )
    
    i+=1
  plt.title(title)
  plt.show()

In [ ]:
df1 = result.groupby("Commune_FR")["quest_id"].nunique().to_frame(name='qty').sort_values(ascending=False, by="qty")
df1["freq"] = df1["qty"]/df1["qty"].sum()
print(f"The three city with most applicants are : \n{df1.head(3)}")
print("We could consider them beacause we suppose that no bootcamp have been already done anywhere.")

generate_barchart(data=df1.head(3),title="Three most valuable communes",abs_value="qty", rel_value='freq')

In [ ]:
def pivotabl(x):
  new_pivot=pd.pivot_table(result, values='quest_id', 
                    index=['Commune_FR'], 
                    columns=x,
                    aggfunc='count')
  new_pivot=new_pivot.fillna(0)
  #new_pivot.drop(["unkwown", "other", "Other"],axis=1, inplace=True)
  return new_pivot

In [ ]:
pv_gender=pivotabl("gender")
pv_edulevel = pivotabl("education_level")
Chanel = pivotabl("chanels")
Project = pivotabl('Project')
pv_internet = pivotabl("internet_at_home")
pv_computer = pivotabl("have_computer_home")
pv_employed = pivotabl('current_employed')
pv_internet.rename(columns={"No":"Don't have internet", "Yes":"Have internet"}, inplace=True)
pv_computer.rename(columns={"No":"Don't have computer", "Yes":"Have computer"}, inplace=True)
pv_employed.rename(columns={"No":"Don't be employed", "Yes":"Employed"}, inplace=True)

In [ ]:
data_commune = pd.concat([pv_gender, pv_edulevel, Chanel, Project, pv_internet, pv_computer, pv_employed], axis=1)
data_commune["qty"]=result.groupby("Commune_FR")["quest_id"].nunique().sort_values()
#data_commune.to_frame(name="qty").sort_values(by=['qty'])
data_commune.sort_values(by=["qty"], inplace=True, ascending=False)
data_commune5 = data_commune[data_commune.qty>5]
print("this dataframe contains the data for each commune with the number of applicants for each variable, it can allow us to see the best prepared communes in number of applicants")
data_commune5

In [ ]:
data_commune5.drop(["unknown", "other", "Other"], axis=1, inplace=True, )
data_commune5.columns


In [ ]:
plt.style.use("ggplot")

In [ ]:
dat_commune = data_commune[["female", "male", "qty"]].reset_index()
#dat_commune.reset_index()
fig, ax = plt.subplots(figsize=(15,5))
dat_commune.head(6).plot.bar(x = 'Commune_FR', y = ['female', 'male', "qty"], rot = 60, ax = ax)
for p in ax.patches: 
    ax.annotate(np.round(p.get_height(),decimals=2), (p.get_x()+p.get_width()/2., p.get_height()))

In [ ]:
pv = data_commune[["qty", "male", "female"]]
pv["women freq"] = (pv["female"]/pv["qty"])
pv["Min_App"] = round(8/pv["women freq"], 0)
pv = pv.reset_index()
pv = pv.head(6)

In [ ]:
for i in [0,1,2,4,5]:
   a= pv.at[i,"Commune_FR"]
   b= pv.at[i,"qty"]
   c= pv.at[i,"female"]
   d= pv.at[i,"Min_App"]
   print(f"In {a}, we have {b} applicants including {c} women, if we want to be sure we have 8 women, 25% of 30 \nWe have to receive at least {d} applicants.")
print("We notice that for Croix-Des-Bouquets, we had 0 woman, so we can predict for how many applicants, we'll have 8 women.")

In [ ]:
#signup_per_week=result5[["Date d'inscription","quest_id"]].groupby(pd.Grouper(key="Date d'inscription",freq='7D')).count()
signup_per_week = result[["signup_date","quest_id", "male","female"]].groupby(pd.Grouper(key="signup_date",freq='7D')).sum()

In [ ]:
signup_per_week["qty"]=result[["signup_date","quest_id", ]].groupby(pd.Grouper(key="signup_date",freq='7D')).count()
signup_per_week["week"] = ["week 1", "week 2", "week 3", "week 4", "week 5"]
fig, ax = plt.subplots(figsize=(15,5))
signup_per_week.plot.line(x = 'week', y = ['female', 'male', "qty"], rot = 60, ax = ax)
for p in ax.patches: 
    ax.annotate(np.round(p.get_height(),decimals=2), (p.get_x()+p.get_width()/2., p.get_height()))
 #montre qu'une prolongation de la date d'inscription ne fait pas augmenter les sign up.

In [ ]:
gend_chan = pd.crosstab(result["gender"], result["chanels"], margins=True)
gend_chan

In [ ]:
plot1 = gend_chan.iloc[2:3, :-1].plot(kind="bar", figsize=(12,5))

In [ ]:
gend_chanels = gend_chan[["Ayiti Analytics Website", "Bootcamp Alumni", "ESIH", "Facebook","Friend","LinkedIn","WhatsApp" ]]
gend_chanels=gend_chanels.to_numpy().T[:,:2]
gend_chanels
from scipy.stats import chi2_contingency
chi2, pval, dof, expected = chi2_contingency(gend_chanels)
print(f"The p=value is {round(pval, 4)}< 0.05, so With a significance level of 0.05,\nwe can therefore conclude that the association between the variables Gender and Channels is statistically significant.")
print("We can propose to see the specific chanels to encourage womwn to apply")

In [ ]:
w = []
m = []
for i in range(10):
  a = (gend_chan.iloc[0:1,i:i+1].to_numpy()).tolist()[0][0]
  w.append(a)
for i in range(10):
  b = (gend_chan.iloc[1:2,i:i+1].to_numpy()).tolist()[0][0]
  m.append(b)
for i in range(9):
  if w[i]>m[i]:
    loc = gend_chan.columns[i]
    print(f"The number of women for the {loc} is bigger tan for men, \nand we should use this chanel to get more women.")

In [ ]:
result["scor1"] = result["have_computer_home"].apply(lambda x: 0.6 if x=="Yes" else 0)
result["scor2"] = result['internet_at_home'].apply(lambda x: 0.4 if x=="Yes" else 0)
result["tot_sc"] = result["scor1"]+result["scor2"]

In [ ]:

result["scor1"] = result["have_computer_home"].apply(lambda x: 0.6 if x=="Yes" else 0)
result["scor2"] = result['internet_at_home'].apply(lambda x: 0.4 if x=="Yes" else 0)
result["tot_sc"] = result["scor1"]+result["scor2"]
In [84]:
df3 = pd.DataFrame()
df3["scor"] = result.groupby("Commune_FR")["tot_sc"].sum().sort_values(ascending=False)
df3["qty"]=result.groupby("Commune_FR")["quest_id"].nunique()
df3 = df3[df3['qty']>5]
df3["moy"]=df3["scor"]/df3["qty"]
df3

In [ ]:

listboth = []
for c in df3.index:
  df = result[result.Commune_FR== c]
  df["both"] = df.have_computer_home.apply(lambda x: 1 if x=="Yes" else 0) + df["internet_at_home"].apply(lambda x: 1 if x=="Yes" else 0)
  g = df["both"].apply(lambda x: 1 if x==2 else 0)
  n = g.sum()
  print(f"{c} has {n} people who have both computer and internet.\n")

In [ ]:
for c in data_commune5.index:
   i = data_commune5.at[c, "qty"]
   j = data_commune5.at[c, "Have internet"]
   o = data_commune5.at[c, "Don't have internet"]
   l = data_commune5.at[c, "Have computer"]
   t = data_commune5.at[c, "Don't have computer"]
   print(f"{c} has {i} applicants, {j} have internet and {o} don't have \n{l} have computer and {t} don't have. \n")